In [11]:
%matplotlib inline

In [12]:
import numpy as np
from scipy.stats import scoreatpercentile
from nilearn.image import iter_img
from nilearn.plotting import plot_stat_map, show
from pylab import figure
import os

from nilearn.datasets import load_mni152_brain_mask
mask_img = load_mni152_brain_mask()

from nilearn.input_data import NiftiMasker
masker = NiftiMasker(mask_img=mask_img, standardize=True)
masker.fit()

def flip(row):
    if np.sum(row > 0) < np.sum(row < 0):
        row *= -1
    return row
        
import statistics
# components_img1=stats.zscore(components_img1,axis=0)
# components_img2=stats.zscore(components_img2,axis=0)
# components_img3=stats.zscore(components_img3,axis=0)
def thresholding(array,P):    
    thr=[]
    array1=array
    
    for idx,row in enumerate(array):
#         row=flip(row)  
        row[row < 0] = 0
        T=np.amax(row)*P
        row[np.abs(row) < T] = 0
        
#         T=scoreatpercentile(np.abs(row),P)
#         T=2.3                
#         T=max(row)*0.2
        
        row=row/np.std(row)        
        array1[idx,:]=row
    return array1

def save_maps(components_img,dir):        
    if not os.path.exists(dir):
        os.makedirs(dir)
    components_img = masker.inverse_transform(components_img)
    for i, cur_img in enumerate(iter_img(components_img)):
        outname=dir+str(i+1)+'.png'
        plot_stat_map(cur_img, display_mode="z", black_bg=True,cut_coords=1,
                     colorbar=False,output_file=outname,title="ICN %d" % int(i))
        
        if i==5:
            break




In [13]:
def vis(task):
    W1=np.load('/home/share/TmpData/Qinglin/HCP_Group_DBN/'+task+'_W1.npy')
    W2=np.load('/home/share/TmpData/Qinglin/HCP_Group_DBN/'+task+'_W2.npy')
    W3=np.load('/home/share/TmpData/Qinglin/HCP_Group_DBN/'+task+'_W3.npy')

    components_img1=W1
    components_img2=np.matmul(W1,W2)
    components_img3=np.matmul(components_img2,W3)

    components_img1=np.transpose(components_img1)
    components_img2=np.transpose(components_img2)
    components_img3=np.transpose(components_img3)
    
    components_img1=thresholding(components_img1,0.3)    
    components_img2=thresholding(components_img2,0.4)
    components_img3=thresholding(components_img3,0.2)

    save_maps(components_img1,dir='/storage/dbn/maps/'+task+'_W1/')        
    save_maps(components_img2,dir='/storage/dbn/maps/'+task+'_W2/')   
    save_maps(components_img3,dir='/storage/dbn/maps/'+task+'_W3/')  
    
#     np.save('/home/share/TmpData/Qinglin/HCP_Group_DBN/'+task+'_map1.npy',components_img1)
#     np.save('/home/share/TmpData/Qinglin/HCP_Group_DBN/'+task+'_map2.npy',components_img2)
#     np.save('/home/share/TmpData/Qinglin/HCP_Group_DBN/'+task+'_map3.npy',components_img3)
    


In [14]:
# from joblib import Parallel, delayed
# tasks=['EMOTION','GAMBLING','LANGUAGE','MOTOR','RELATIONAL','SOCIAL','WM']
# Parallel(n_jobs=7)(delayed(vis)(task) for task in tasks)
vis('EMOTION')
  